# Defining data types

We can define types (i.e. data structures) ourselves using the `struct` keyword.

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case).

(Note that types can not be redefined - you have to restart your Julia session to change a type definiton.)

In [1]:
struct MyType end

To create an object of type `MyType` we have to call a [constructor](https://docs.julialang.org/en/latest/manual/constructors/). Loosely speaking, a constructor is a function that create new objects.

Julia automatically creates a trivial constructors for us, which has the same name as the type.

In [2]:
methods(MyType)

# 1 method for generic function "(::Type)":
[1] MyType() in Main at In[1]:1

In [3]:
m = MyType()

MyType()

In [4]:
typeof(m)

MyType

In [5]:
m isa MyType

true

Since no data is contained in our `MyType`  - it is a so-called *singleton type* - we can basically only use it for dispatch.

Most of the time, we'll want a self-defined type to hold some data. For this, we need *fields*.

In [6]:
struct A
    x::Int64
end

In [7]:
A()

MethodError: MethodError: no method matching A()
Closest candidates are:
  A(!Matched::Int64) at In[6]:2
  A(!Matched::Any) at In[6]:2

The default constructor always expects values for all fields.

In [8]:
A(3)

A(3)

In [9]:
a = A(3)

A(3)

In [10]:
# a.<TAB>
a.x

3

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [11]:
a.x = 2

ErrorException: setfield! immutable struct of type A cannot be changed

In [12]:
mutable struct B
    x::Int64
end

In [13]:
b = B(3)

B(3)

In [14]:
b.x

3

In [15]:
b.x = 4

4

In [16]:
b.x

4

Note, however, that **immutability is not recursive**.

In [17]:
struct C
    x::Vector{Int64}
end

In [18]:
c = C([1, 2, 3])

C([1, 2, 3])

In [19]:
c.x

3-element Array{Int64,1}:
 1
 2
 3

In [20]:
c.x = [3,4,5]

ErrorException: setfield! immutable struct of type C cannot be changed

In [21]:
c.x[1] = 3

3

In [22]:
c.x

3-element Array{Int64,1}:
 3
 2
 3

In [23]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

3-element Array{Int64,1}:
 3
 4
 5

Abstract types are just as easy to define using the keyword `abstract type`.

In [24]:
abstract type MyAbstractType end

Since abstract types don't have fields, they only (informally) define interfaces and can be used for dispatch.

In [25]:
struct MyConcreteType <: MyAbstractType # subtype
    somefield::String
end

In [26]:
c = MyConcreteType("test")

MyConcreteType("test")

In [27]:
c isa MyAbstractType

true

In [28]:
supertype(MyConcreteType)

MyAbstractType

In [29]:
subtypes(MyAbstractType)

1-element Array{Any,1}:
 MyConcreteType

# Custom constructor

In [30]:
struct VolNaive
    value::Float64
end

In [31]:
VolNaive(3.0)

VolNaive(3.0)

In [32]:
VolNaive(-3.0)

VolNaive(-3.0)

In [33]:
struct VolSimple
    value::Float64
    
    function VolSimple(x) # inner constructor. function name must match the type name.
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new(x) # within an inner constructor, the `new` function can be used to create an object.
    end
end

---

**Side note:**

```julia
if !(x isa Real)
    throw(ArgumentError("Must be real"))
end
if x < 0
    throw(ArgumentError("Negative volume not allowed."))
end
```

This can be written more compactly as
```julia
x isa Real || throw(ArgumentError("Must be real"))
x < 0 && throw(ArgumentError("Negative volume not allowed."))
```

See ["short-circuit evaluation"](https://docs.julialang.org/en/latest/manual/control-flow/#Short-Circuit-Evaluation-1) for more information.

---

In [34]:
VolSimple(3.0)

VolSimple(3.0)

In [35]:
VolSimple(-3.0)

ArgumentError: ArgumentError: Negative volume not allowed.

In [36]:
VolSimple("test")

ArgumentError: ArgumentError: Must be real

In [37]:
VolSimple(3) # implicit conversion from Int64 -> Float64

VolSimple(3.0)

# Parametric types

Volumes don't have to be `Float64` values. We can easily relax our type definition to allow all sorts of internal value types.

In [38]:
struct VolParam{T}
    value::T
    
    function VolParam(x::T) where T # x can be of any type T
        if !(x isa Real)
            throw(ArgumentError("Must be real"))
        end
        if x < 0
            throw(ArgumentError("Negative volume not allowed."))
        end
        new{T}(x) # Note that we need an extra {T} here
    end
end

In [39]:
VolParam(3.0)

VolParam{Float64}(3.0)

In [40]:
VolParam(3)

VolParam{Int64}(3)

Instead of checking the realness of the input `x` explicitly in the inner constructor, we can impose type constraints in the type and function signatures.

In [41]:
struct Vol{T<:Real} <: Real # the last <: Real tells Julia that a Vol is a subtype of Real, i.e. basically a real number
    value::T
    
    function Vol(x::T) where T<:Real # x can be of any type T<:Real
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        new{T}(x)
    end
end

In [42]:
Vol(3)

Vol{Int64}(3)

In [43]:
Vol(3.0)

Vol{Float64}(3.0)

In [44]:
Vol("1.23")

MethodError: MethodError: no method matching Vol(::String)
Closest candidates are:
  Vol(!Matched::Complex) where T<:Real at complex.jl:37
  Vol(!Matched::T<:Real) where T<:Real at In[41]:5
  Vol(!Matched::T<:Number) where T<:Number at boot.jl:718
  ...

In [45]:
Vol(-2)

ArgumentError: ArgumentError: Negative volume not allowed.

# Arithmetic

In [46]:
Vol(3) + Vol(4)

ErrorException: + not defined for Vol{Int64}

In [47]:
+(x::Vol, y::Vol) = Vol(x.value + y.value)

ErrorException: error in method definition: function Base.+ must be explicitly imported to be extended

If we want to extend or override functions that already exit, we need to `import` them first.

In [48]:
import Base: +

+(x::Vol, y::Vol) = Vol(x.value + y.value)

+ (generic function with 162 methods)

In [49]:
Vol(3) + Vol(4)

Vol{Int64}(7)

In [50]:
Vol(2) + Vol(8.3) # implicit conversion!

Vol{Float64}(10.3)

In [51]:
methodswith(Vol)

[1] +(x::Vol, y::Vol) in Main at In[48]:3

In [52]:
import Base: -, *

-(x::Vol, y::Vol) = Vol(x.value - y.value)
*(x::Vol, y::Vol) = Vol(x.value * y.value)

* (generic function with 355 methods)

Now that we have addition defined for our volume type, some functions already **just work**.

In [53]:
sum([Vol(3), Vol(4.8), Vol(1)])

Vol{Float64}(8.8)

In [54]:
M = Vol.(rand(3,3))

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.0625207)  Vol{Float64}(0.782219)  Vol{Float64}(0.947736)
 Vol{Float64}(0.979332)   Vol{Float64}(0.97025)   Vol{Float64}(0.938385)
 Vol{Float64}(0.509278)   Vol{Float64}(0.367216)  Vol{Float64}(0.15844) 

In [55]:
N = Vol.(rand(3,3))

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.379941)  Vol{Float64}(0.746793)   Vol{Float64}(0.288604)
 Vol{Float64}(0.86375)   Vol{Float64}(0.0163442)  Vol{Float64}(0.845622)
 Vol{Float64}(0.365665)  Vol{Float64}(0.0202836)  Vol{Float64}(0.324366)

In [56]:
M + N

3×3 Array{Vol{Float64},2}:
 Vol{Float64}(0.442462)  Vol{Float64}(1.52901)   Vol{Float64}(1.23634) 
 Vol{Float64}(1.84308)   Vol{Float64}(0.986594)  Vol{Float64}(1.78401) 
 Vol{Float64}(0.874943)  Vol{Float64}(0.3875)    Vol{Float64}(0.482806)

Whenever something doesn't work, we implement the necessary functions.

In [57]:
sin(Vol(3))

MethodError: MethodError: no method matching AbstractFloat(::Vol{Int64})
Closest candidates are:
  AbstractFloat(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:194
  AbstractFloat(::T<:Number) where T<:Number at boot.jl:718
  AbstractFloat(!Matched::Bool) at float.jl:252
  ...

In [58]:
import Base: AbstractFloat
AbstractFloat(x::Vol{T}) where T = AbstractFloat(x.value)

AbstractFloat

In [59]:
sin(Vol(3))

0.1411200080598672

In [60]:
sqrt(Vol(3))

1.7320508075688772

If we really wanted to have `Vol{T}` objects behave like real numbers in all operations, we'd have to do a bit more work like specifying [promotion and conversion rules](https://docs.julialang.org/en/latest/manual/conversion-and-promotion/).

An important thing to note is that **user defined types are just as good as built-in types**!

There is nothing special about built-in types. In fact, [they are implemented in the same way](https://github.com/JuliaLang/julia/blob/master/stdlib/LinearAlgebra/src/diagonal.jl#L5)!

Let us quickly confirm that our volume "wrapper" type does not come with any performance overhead by benchmarking it in a simple function.

# Benchmarking with `BenchmarkTools.jl`

In [61]:
using BenchmarkTools

In [62]:
operation(x) = x^2 + sqrt(x)

operation (generic function with 1 method)

In [63]:
x = rand(2,2)
@time operation.(x)

  0.117131 seconds (209.45 k allocations: 10.382 MiB)


2×2 Array{Float64,2}:
 1.73048  1.00642
 1.75946  1.12159

In [64]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

f (generic function with 1 method)

In [65]:
f()

  0.000000 seconds (1 allocation: 112 bytes)


2×2 Array{Float64,2}:
 0.671089  1.17494 
 1.40787   0.306345

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [66]:
@benchmark operation.(x)

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  3
  --------------
  minimum time:     328.179 ns (0.00% GC)
  median time:      348.908 ns (0.00% GC)
  mean time:        426.442 ns (10.24% GC)
  maximum time:     279.385 μs (99.84% GC)
  --------------
  samples:          10000
  evals/sample:     223

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [67]:
@btime operation.(x);

  341.502 ns (3 allocations: 144 bytes)


However, we still have to take some care to avoid accessing global variables.

In [68]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

  43.593 ns (1 allocation: 112 bytes)


More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).

Finally, we can check the performance of our custom volume type.

In [69]:
@btime sqrt(Vol(3));
@btime sqrt(3);

  1.549 ns (0 allocations: 0 bytes)
  1.763 ns (0 allocations: 0 bytes)


# Core messages of this Notebook

* There are `mutable struct`s and immutable `struct`s and immutability is not recursive.
* **Contructors** are functions that create objects. In an inner constructor we can use the function `new` to generate objects.
* We can easily **extend `Base` functions** for our types to implement arithmetics and such.
* We should always benchmark our code with **BenchmarkTools.jl's @btime and @benchmark**.